In [10]:
import numpy as np
import pandas as df
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import train_test_split

positive_dir = Path('/Users/laura/Downloads/concerete_crack_images/training/Positive/')
negative_dir = Path('/Users/laura/Downloads/concerete_crack_images/training/Negative/')
test_data = Path('/Users/laura/Downloads/concerete_crack_images/testing/')

In [11]:
import numpy as np
import pandas as df
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import train_test_split

positive_dir = Path('/Users/laura/Downloads/concerete_crack_images/training/Positive/')
negative_dir = Path('/Users/laura/Downloads/concerete_crack_images/training/Negative/')

In [12]:
def generate_df(image_dir, label):
    filepaths = pd.Series(list(image_dir.glob(r'*.jpg')), name='Filepath').astype(str)
    labels = pd.Series(label, name='Label', index=filepaths.index)
    df = pd.concat([filepaths, labels], axis=1)
    return df


In [13]:
positive_df = generate_df(positive_dir, label="POSITIVE")
negative_df = generate_df(negative_dir, label="NEGATIVE")

df = pd.concat([positive_df, negative_df], axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
df

,Filepath,Label
0,/Users/laura/Downloads/concerete_crack_images/...,NEGATIVE
1,/Users/laura/Downloads/concerete_crack_images/...,POSITIVE
2,/Users/laura/Downloads/concerete_crack_images/...,POSITIVE
3,/Users/laura/Downloads/concerete_crack_images/...,NEGATIVE
4,/Users/laura/Downloads/concerete_crack_images/...,NEGATIVE
...,...,...
32102,/Users/laura/Downloads/concerete_crack_images/...,NEGATIVE
32103,/Users/laura/Downloads/concerete_crack_images/...,POSITIVE
32104,/Users/laura/Downloads/concerete_crack_images/...,POSITIVE
32105,/Users/laura/Downloads/concerete_crack_images/...,POSITIVE


---test1---

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam
img_size = 150
model_name = 'ResNet50'


pre_model = ResNet50(weights='imagenet', include_top=False,
                   input_shape=(img_size, img_size, 3))

pre_model.trainable = False

x = layers.GlobalAveragePooling2D()(pre_model.output)

outputs = layers.Dense(1, activation='sigmoid')(x)
model = Model(inputs=pre_model.inputs, outputs=outputs)

optimizer = Adam(learning_rate=0.001)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

94765736/94765736 [==============================] - 6s 0us/step


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
               width_shift_range=0.1,  
               height_shift_range=0.1,  
               horizontal_flip=True,  
               preprocessing_function=preprocess_input,  
               validation_split=0.01  
              )

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.01)

In [4]:

img_shape = (227, 227)


train_generator = train_datagen.flow_from_directory(
                 '/Users/laura/Downloads/concerete_crack_images',
                 target_size=(227, 227),  
                 batch_size=64,  
                 shuffle=True,  
                 class_mode='binary',  
                 subset='training')  

validation_generator =  validation_datagen.flow_from_directory(
                        '/Users/laura/Downloads/concerete_crack_images',
                        target_size=(227, 227),  
                        batch_size=64,  
                        class_mode='binary',  
                        subset='validation')  

Found 39601 images belonging to 2 classes.
Found 399 images belonging to 2 classes.


In [7]:

model.fit(train_generator,
          epochs=3,
          validation_data=validation_generator)

Epoch 1/3
619/619 [==============================] - 1822s 3s/step - loss: 0.5128 - accuracy: 0.8013 - val_loss: 0.5062 - val_accuracy: 0.8045
Epoch 2/3
619/619 [==============================] - 1944s 3s/step - loss: 0.5061 - accuracy: 0.8027 - val_loss: 0.5064 - val_accuracy: 0.8045
Epoch 3/3
619/619 [==============================] - 1498s 2s/step - loss: 0.5049 - accuracy: 0.8026 - val_loss: 0.5116 - val_accuracy: 0.8045


---test1---

In [14]:
train_df, test_df = train_test_split(df,train_size=0.7,shuffle=True,random_state=100)
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.2)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(227, 227),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training')

val_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(227, 227),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation')

test_data = train_gen.flow_from_dataframe(
    test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(227, 227),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,
    seed=42)


Found 17980 validated image filenames belonging to 2 classes.
Found 4494 validated image filenames belonging to 2 classes.
Found 9633 validated image filenames belonging to 2 classes.


In [15]:
inputs = tf.keras.Input(shape=(227, 227, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 225, 225, 16)      448       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 112, 112, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 110, 110, 32)      4640      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 55, 55, 32)        0         
 g2D)                                                            
                                                                 
 global_average_pooling2d_1  (None, 32)                0   

In [16]:
history = model.fit(train_data,validation_data=val_data,epochs=10,
            callbacks=[tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True)])

Epoch 1/10
562/562 [==============================] - 139s 247ms/step - loss: 0.5999 - accuracy: 0.6981 - val_loss: 0.4469 - val_accuracy: 0.8718
Epoch 2/10
562/562 [==============================] - 135s 240ms/step - loss: 0.3112 - accuracy: 0.9162 - val_loss: 0.2226 - val_accuracy: 0.9359
Epoch 3/10
562/562 [==============================] - 120s 214ms/step - loss: 0.1986 - accuracy: 0.9445 - val_loss: 0.1743 - val_accuracy: 0.9417
Epoch 4/10
562/562 [==============================] - 120s 214ms/step - loss: 0.1599 - accuracy: 0.9555 - val_loss: 0.1414 - val_accuracy: 0.9626
Epoch 5/10
562/562 [==============================] - 123s 219ms/step - loss: 0.1422 - accuracy: 0.9592 - val_loss: 0.1243 - val_accuracy: 0.9608
Epoch 6/10
562/562 [==============================] - 129s 229ms/step - loss: 0.1231 - accuracy: 0.9657 - val_loss: 0.1454 - val_accuracy: 0.9637
Epoch 7/10
562/562 [==============================] - 129s 230ms/step - loss: 0.1197 - accuracy: 0.9674 - val_loss: 0.1011 -

In [17]:

predictions = model.predict(test_data)

binary_predictions = (predictions > 0.5).astype(int)

submission_df = pd.DataFrame({'image_id': test_data.filenames, 'predicted_class': binary_predictions.squeeze()})
submission_df['image_id'] = submission_df['image_id'].apply(lambda x: x.split('/')[1])

submission_df.to_csv('submission.csv', index=False)


302/302 [==============================] - 22s 71ms/step
